<a href="https://colab.research.google.com/github/RAVIPATISRIVIDYA/devtraining-needit-madrid/blob/master/Poisonattack1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 5.6 MB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
     |████████████████████████████████| 26.3 MB 83 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [2]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from art.utils import load_dataset
from art.utils import load_dataset
from art.attacks.poisoning.perturbations.image_perturbations import add_pattern_bd, add_single_bd
from art.utils import load_mnist, preprocess,load_cifar10
from art.estimators.classification import KerasClassifier
from __future__ import absolute_import, division, print_function, unicode_literals
import art
import pprint
import json

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np

In [7]:
def generate_backdoor(
    x_clean, y_clean, num_poison, backdoor_type="pattern", sources=np.arange(10), targets=(np.arange(10) + 1) % 10
):

    max_val = np.max(x_clean)

    x_poison = np.copy(x_clean)
    y_poison = np.copy(y_clean)
    is_poison = np.zeros(np.shape(y_poison))

    for i, (src, tgt) in enumerate(zip(sources, targets)):
      n_points_in_tgt = np.size(np.where(y_clean == tgt))
      num_poison = num_poison #round((percent_poison * n_points_in_tgt) / (1 - percent_poison))
      print(num_poison)
      
      src_imgs = x_clean[np.ravel(y_clean == src)]
      # src_imgs = x_clean[y_clean == src]

      n_points_in_src = np.shape(src_imgs)[0]
      indices_to_be_poisoned = np.random.choice(n_points_in_src, num_poison)

      imgs_to_be_poisoned = np.copy(src_imgs[indices_to_be_poisoned])
      if backdoor_type == "pattern":
          imgs_to_be_poisoned = add_pattern_bd(x=imgs_to_be_poisoned, pixel_value=max_val)
      elif backdoor_type == "pixel":
          imgs_to_be_poisoned = add_single_bd(imgs_to_be_poisoned, pixel_value=max_val)
      x_poison = np.append(x_poison, imgs_to_be_poisoned, axis=0)
      # import pdb
      # pdb.set_trace()
      y_poison = np.append(y_poison, np.ones((num_poison,1)) * tgt, axis=0)
      # y_poison = np.append(y_poison, np.ones(num_poison) * tgt, axis=0)
      is_poison = np.append(is_poison, np.ones(num_poison))

    is_poison = is_poison != 0

    return is_poison, x_poison, y_poison


In [ ]:
max_val = np.max(x_clean)

    x_poison = np.copy(x_clean)
    y_poison = np.copy(y_clean)
    is_poison = np.zeros(np.shape(y_poison))

    for i, (src, tgt) in enumerate(zip(sources, targets)):
      n_points_in_tgt = np.size(np.where(y_clean == tgt))
      num_poison = num_poison #round((percent_poison * n_points_in_tgt) / (1 - percent_poison))
      print(num_poison)
      
      src_imgs = x_clean[np.ravel(y_clean == src)]
      # src_imgs = x_clean[y_clean == src]

      n_points_in_src = np.shape(src_imgs)[0]
      indices_to_be_poisoned = np.random.choice(n_points_in_src, num_poison)

      imgs_to_be_poisoned = np.copy(src_imgs[indices_to_be_poisoned])
      if backdoor_type == "pattern":
          imgs_to_be_poisoned = add_pattern_bd(x=imgs_to_be_poisoned, pixel_value=max_val)
      elif backdoor_type == "pixel":
          imgs_to_be_poisoned = add_single_bd(imgs_to_be_poisoned, pixel_value=max_val)
      x_poison = np.append(x_poison, imgs_to_be_poisoned, axis=0)
      # import pdb
      # pdb.set_trace()
      y_poison = np.append(y_poison, np.ones((num_poison,1)) * tgt, axis=0)
      # y_poison = np.append(y_poison, np.ones(num_poison) * tgt, axis=0)
      is_poison = np.append(is_poison, np.ones(num_poison))

    is_poison = is_poison != 0

    return is_poison, x_poison, y_poison

In [14]:
(x_raw, y_raw), (x_raw_test, y_raw_test), min_, max_ = load_cifar10(raw=True)
n_train = np.shape(x_raw)[0]
num_selection = 50000
random_selection_indices = np.random.choice(n_train, num_selection)
x_raw = x_raw[random_selection_indices]
y_raw = y_raw[random_selection_indices]
print(x_raw.shape,y_raw.shape)
# num_images_to_poison= 2000
perc_poison= 0.04#num_images_to_poison/x_raw.shape[0]
(is_poison_train, x_poisoned_raw, y_poisoned_raw) = generate_backdoor(x_raw, y_raw, num_poison=1500)
# x_train, y_train = preprocess(x_poisoned_raw, y_poisoned_raw)
x_train,y_train = x_poisoned_raw/255,to_categorical(y_poisoned_raw,10)
# Add channel axis:
# x_train = np.expand_dims(x_train, axis=3)
# Poison test data
(is_poison_test, x_poisoned_raw_test, y_poisoned_raw_test) = generate_backdoor(x_raw_test, y_raw_test, num_poison=100)
# x_test, y_test = preprocess(x_poisoned_raw_test, y_poisoned_raw_test)
x_test,y_test = x_poisoned_raw_test/255,to_categorical(y_poisoned_raw_test,10)
# Add channel axis:
# x_test = np.expand_dims(x_test, axis=3)
# Shuffle training data so poison is not together
n_train = np.shape(y_train)[0]
shuffled_indices = np.arange(n_train)
np.random.shuffle(shuffled_indices)
x_train = x_train[shuffled_indices]
y_train = y_train[shuffled_indices]
is_poison_train = is_poison_train[shuffled_indices]

(50000, 32, 32, 3) (50000, 1)
1500
1500
1500
1500
1500
1500
1500
1500
1500
1500
100
100
100
100
100
100
100
100
100
100


In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

classifier = KerasClassifier(model=model, clip_values=(min_, max_))

classifier.fit(x_train, y_train, nb_epochs=30, batch_size=128)

Train on 65000 samples
Epoch 1/30
65000/65000 [==============================] - 11s 167us/sample - loss: 1.7634 - accuracy: 0.3481
Epoch 2/30
65000/65000 [==============================] - 11s 164us/sample - loss: 1.3969 - accuracy: 0.4949
Epoch 3/30
65000/65000 [==============================] - 11s 164us/sample - loss: 1.2361 - accuracy: 0.5614
Epoch 4/30
65000/65000 [==============================] - 11s 163us/sample - loss: 1.1319 - accuracy: 0.5971
Epoch 5/30
65000/65000 [==============================] - 11s 164us/sample - loss: 1.0491 - accuracy: 0.6280
Epoch 6/30
65000/65000 [==============================] - 11s 164us/sample - loss: 0.9797 - accuracy: 0.6512
Epoch 7/30
65000/65000 [==============================] - 11s 162us/sample - loss: 0.9122 - accuracy: 0.6760
Epoch 8/30
65000/65000 [==============================] - 11s 163us/sample - loss: 0.8654 - accuracy: 0.6904
Epoch 9/30
65000/65000 [==============================] - 11s 162us/sample - loss: 0.8238 - accuracy: 0.7

In [16]:
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
print("\nTest accuracy: %.2f%%" % (acc * 100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,



Test accuracy: 67.95%


In [17]:
preds = np.argmax(classifier.predict(x_test[is_poison_test]), axis=1)
acc = np.sum(preds == np.argmax(y_test[is_poison_test], axis=1)) / y_test[is_poison_test].shape[0]
print("\nPoisonous test set accuracy (i.e. effectiveness of poison): %.2f%%" % (acc * 100))


Poisonous test set accuracy (i.e. effectiveness of poison): 64.60%


In [18]:
preds = np.argmax(classifier.predict(x_test[is_poison_test == 0]), axis=1)
acc = np.sum(preds == np.argmax(y_test[is_poison_test == 0], axis=1)) / y_test[is_poison_test == 0].shape[0]
print("\nClean test set accuracy: %.2f%%" % (acc * 100))


Clean test set accuracy: 68.28%
